In [1]:
import pandas as pd
import itertools
import os
import pandas.io.formats.style

In [2]:
pd.options.display.float_format = "{:,.3f}".format

### Globals, Helper Functions, etc

In [3]:
MEASURE_NAME_DICT = {
    "AlignedCosineSimilarity": "AlignCos",
    "CKA": "CKA",
    "ConcentricityDifference": "ConcDiff",
    "DistanceCorrelation": "DistCorr",
    "EigenspaceOverlapScore": "EOS",
    "GeometryScore": "GS",
    "Gulp": "GULP",
    "HardCorrelationMatch": "HardCorr",
    "IMDScore": "IMD",
    "JaccardSimilarity": "Jaccard",
    "LinearRegression": "LinReg",
    "MagnitudeDifference": "MagDiff",
    "OrthogonalAngularShapeMetricCentered": "ShapeMet",
    "OrthogonalProcrustesCenteredAndNormalized": "OrthProc",
    "PWCCA": "PWCCA",
    "PermutationProcrustes": "PermProc",
    "ProcrustesSizeAndShapeDistance": "ProcDist",
    "RSA": "RSA",
    "RSMNormDifference": "RSMDiff",
    "RankSimilarity": "RankSim",
    "SVCCA": "SVCCA",
    "SecondOrderCosineSimilarity": "2nd-Cos",
    "SoftCorrelationMatch": "SoftCorr",
    "UniformityDifference": "UnifDiff",
}

PIVOT_COL_DICT = {
    "cora": "Cora",
    "flickr": "Flickr",
    "ogbn-arxiv": "OGBN-Arxiv",
    "GraphSAGE": "SAGE",
    "violation_rate": "Violation Rate",
    "correlation": "Spearman Correlation"
}

COLUMN_NAME_DICT = {
    "similarity_measure": "Similarity Measure", 
    "quality_measure": "Measure",
    "functional_similarity_measure": "Measure",
    "architecture": "Model",
    "representation_dataset": "Dataset"
}

LATEX_FORMAT_DICT = {
    "hrules": True,
    "column_format": "l||rrr|rrr|rrr||rrr|rrr|rrr",
    "multicol_align": "c",
}

LATEX_FORMAT_DICT_PGNN = {
    "hrules": True,
    "column_format": "l||rrrr|rrr|rrr||rrrr|rrr|rrr",
    "multicol_align": "c",
}

In [4]:
EXPERIMENT_RESULTS_PATH = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "experiments", "results", "full_csvs")

def FULL_DF_FILE_NAME(experiment, dataset):
    return f"graphs_{experiment}_{dataset}_full.csv"

In [5]:
def get_agg_pivot_table(experiment, datasets, conformity=True):
    dfs = []
    full_csvs = os.listdir(EXPERIMENT_RESULTS_PATH)

    for dataset in datasets:
        for f in full_csvs:
            if experiment in f and dataset in f:
                df = pd.read_csv(os.path.join(EXPERIMENT_RESULTS_PATH, f), encoding="utf-8")
                data = df.loc[:, ["similarity_measure", "quality_measure", "value", "architecture", "representation_dataset"]].dropna()
                data = data.rename(COLUMN_NAME_DICT, axis="columns")
                dfs.append(data.iloc[:])
    df_cc = pd.concat(dfs, axis=0)
    df_cc = df_cc.astype({'Model' : pd.CategoricalDtype(['GCN', 'GraphSAGE', 'GAT', 'PGNN'], ordered = True)})
    df_res = df_cc.pivot_table(index="Similarity Measure", columns=["Measure", "Dataset", "Model"], values="value", aggfunc="mean", observed=False)
    
    if conformity:
        df_res["violation_rate"] = 1 - df_res["violation_rate"]
        df_res = df_res.rename({"violation_rate": "Conformity Rate"}, axis=1)
    df_res = df_res.rename(MEASURE_NAME_DICT, axis="index")
    return df_res.rename(PIVOT_COL_DICT, axis="columns")

### Results for Group Separation Experiments

#### Random Label Test Results

In [6]:
get_agg_pivot_table("label_test", ["cora", "flickr", "ogbn-arxiv"])

Measure            AUPRC                                                  \
Dataset             Cora                   Flickr             OGBN-Arxiv   
Model                GCN  SAGE   GAT  PGNN    GCN  SAGE   GAT        GCN   
Similarity Measure                                                         
AlignCos           0.457 0.479 0.433 0.233  0.839 0.423 0.286      0.981   
CKA                0.427 0.424 0.424 0.247  0.733 0.657 0.270      1.000   
ConcDiff           0.203 0.391 0.394 0.187  0.366 0.570 0.208      0.964   
DistCorr           0.431 0.444 0.424 0.255  0.860 0.427 0.217      1.000   
EOS                0.295 0.259 0.284 0.245  0.413 0.424 0.219      0.253   
GULP               0.276 0.260 0.285 0.263  0.192 0.424 0.227      0.302   
HardCorr           0.425 0.424 0.424 0.226  0.771 0.457 0.332      0.828   
IMD                0.729 0.978 0.839 0.239  0.236 0.369 0.228      1.000   
Jaccard            0.425 0.424 0.427 0.258  0.564 0.430 0.292      0.827   
LinReg             0.492 0.431 0.456 0.235  0.223 0.449 0.230      0.444   
MagDiff            0.199 0.243 0.370 0.282  0.661 0.718 0.185      0.551   
ShapeMet           0.426 0.424 0.424 0.236  0.882 0.434 0.275      0.826   
OrthProc           0.426 0.424 0.424 0.236  0.882 0.434 0.275      0.826   
PWCCA              0.265 0.325 0.277 0.244    NaN 0.444 0.243      0.241   
PermProc           0.443 0.386 0.435 0.284  0.772 0.904 0.192      0.678   
ProcDist           0.452 0.451 0.433 0.284  0.785 0.625 0.190      0.915   
RSA                0.457 0.435 0.425 0.241  0.737 0.424 0.334      0.961   
RSMDiff            0.518 0.527 0.779 0.299  0.715 0.915 0.189      0.997   
RankSim            0.452 0.423 0.475 0.208  0.470 0.426 0.333      0.844   
SVCCA              0.347 0.306 0.422 0.194  0.332 0.796 0.268      1.000   
2nd-Cos            0.559 0.735 0.733 0.224  0.611 0.423 0.365      0.994   
SoftCorr           0.425 0.424 0.423 0.233  0.603 0.449 0.332      0.828   
UnifDiff           0.334 0.497 0.320 0.247  0.531 0.902 0.396      0.531   

Measure                        Conformity Rate                                 \
Dataset                                   Cora                   Flickr         
Model               SAGE   GAT             GCN  SAGE   GAT  PGNN    GCN  SAGE   
Similarity Measure                                                              
AlignCos           0.699 0.931           0.659 0.664 0.601 0.552  0.936 0.500   
CKA                1.000 1.000           0.549 0.512 0.507 0.569  0.907 0.861   
ConcDiff           1.000 0.997           0.545 0.781 0.739 0.430  0.723 0.848   
DistCorr           1.000 1.000           0.590 0.631 0.521 0.644  0.948 0.558   
EOS                0.426 0.340           0.649 0.551 0.658 0.562  0.500 0.516   
GULP               0.426 0.303           0.631 0.556 0.659 0.501  0.503 0.512   
HardCorr           0.828 0.544           0.527 0.511 0.508 0.526  0.941 0.679   
IMD                1.000 0.898           0.929 0.994 0.961 0.531  0.469 0.698   
Jaccard            0.432 0.778           0.527 0.508 0.553 0.569  0.771 0.579   
LinReg             0.677 0.467           0.713 0.579 0.688 0.510  0.489 0.663   
MagDiff            0.490 0.343           0.512 0.604 0.737 0.614  0.892 0.934   
ShapeMet           0.726 0.767           0.539 0.513 0.522 0.584  0.952 0.604   
OrthProc           0.726 0.767           0.539 0.513 0.522 0.584  0.952 0.604   
PWCCA              0.436 0.362           0.601 0.518 0.644 0.531    NaN 0.638   
PermProc           0.724 0.931           0.645 0.714 0.609 0.488  0.939 0.969   
ProcDist           0.983 1.000           0.673 0.677 0.597 0.482  0.932 0.882   
RSA                0.430 0.490           0.693 0.615 0.539 0.563  0.885 0.525   
RSMDiff            1.000 1.000           0.732 0.831 0.863 0.610  0.911 0.971   
RankSim            0.552 0.779           0.655 0.535 0.643 0.475  0.663 0.530   
SVCCA              1.000 0.932           0.619 0.555 0.591 0.383  0.689 0.913   
2nd

In [7]:
label_test_df = get_agg_pivot_table("label_test", ["cora", "flickr", "ogbn-arxiv"])
styled = pd.io.formats.style.Styler(
    label_test_df,
    precision=2,
)

latex_str = styled.highlight_max(axis=0, props="textbf:--rwrap;").to_latex("graphs_results_label_test.tex", **LATEX_FORMAT_DICT_PGNN)

#### Shortcut Test Results

In [8]:
get_agg_pivot_table("shortcut_test", ["cora", "flickr", "ogbn-arxiv"])

Measure            AUPRC                                                  \
Dataset             Cora             Flickr             OGBN-Arxiv         
Model                GCN  SAGE   GAT    GCN  SAGE   GAT        GCN  SAGE   
Similarity Measure                                                         
AlignCos           0.635 0.599 0.588  0.895 1.000 0.480      1.000 1.000   
CKA                0.614 0.777 0.784  0.282 1.000 0.330      1.000 0.981   
ConcDiff           0.512 0.169 0.204  0.182 0.176 0.324      0.815 0.962   
DistCorr           0.661 0.820 0.802  0.333 1.000 0.317      1.000 0.994   
EOS                0.301 0.495 0.197  0.461 0.433 0.544      1.000 0.724   
GULP               0.310 0.496 0.197  0.196 0.433 0.451      0.986 0.724   
HardCorr           0.277 0.347 0.294  0.549 1.000 0.519      0.798 0.724   
IMD                0.719 0.792 0.597  0.783 0.971 0.357      0.609 0.918   
Jaccard            0.736 0.782 0.864  1.000 0.828 0.545      1.000 1.000   
LinReg             0.699 0.742 0.502  0.365 0.609 0.357      1.000 1.000   
MagDiff            0.366 0.151 0.181  0.266 0.783 0.198      0.550 0.535   
ShapeMet           0.629 0.794 0.619  0.581 1.000 0.307      1.000 1.000   
OrthProc           0.629 0.794 0.619  0.581 1.000 0.307      1.000 1.000   
PWCCA              0.328 0.468 0.215    NaN 0.433 0.329      0.990 0.724   
PermProc           0.230 0.268 0.246  0.418 1.000 0.272      0.649 0.433   
ProcDist           0.603 0.812 0.676  0.599 1.000 0.331      1.000 0.997   
RSA                0.490 0.432 0.726  0.894 1.000 0.521      0.966 0.978   
RSMDiff            0.462 0.497 0.689  0.476 0.920 0.362      1.000 1.000   
RankSim            0.732 0.645 0.849  1.000 0.774 0.538      1.000 1.000   
SVCCA              0.232 0.357 0.458  0.239 0.929 0.315      1.000 0.970   
2nd-Cos            0.846 0.907 0.807  1.000 1.000 0.522      1.000 1.000   
SoftCorr           0.306 0.404 0.302  0.663 1.000 0.571      0.816 0.724   
UnifDiff           0.296 0.637 0.712  0.593 0.499 0.480      0.991 0.717   

Measure                  Conformity Rate                                 \
Dataset                             Cora             Flickr               
Model                GAT             GCN  SAGE   GAT    GCN  SAGE   GAT   
Similarity Measure                                                        
AlignCos           1.000           0.840 0.881 0.893  0.981 1.000 0.769   
CKA                1.000           0.752 0.903 0.943  0.572 1.000 0.652   
ConcDiff           1.000           0.775 0.427 0.541  0.497 0.463 0.607   
DistCorr           1.000           0.835 0.916 0.951  0.687 1.000 0.718   
EOS                0.968           0.707 0.740 0.461  0.872 0.600 0.809   
GULP               0.965           0.718 0.739 0.463  0.492 0.600 0.811   
HardCorr           0.835           0.654 0.768 0.695  0.829 1.000 0.808   
IMD                0.916           0.928 0.957 0.881  0.945 0.993 0.621   
Jaccard            1.000           0.915 0.929 0.967  1.000 0.963 0.840   
LinReg             1.000           0.878 0.947 0.755  0.703 0.811 0.723   
MagDiff            1.000           0.745 0.415 0.489  0.669 0.886 0.477   
ShapeMet           1.000           0.787 0.915 0.903  0.798 1.000 0.721   
OrthProc           1.000           0.787 0.915 0.903  0.798 1.000 0.721   
PWCCA              0.997           0.735 0.711 0.561    NaN 0.600 0.735   
PermProc           0.766           0.566 0.672 0.641  0.719 1.000 0.576   
ProcDist           1.000           0.766 0.913 0.920  0.815 1.000 0.608   
RSA                0.902           0.717 0.784 0.914  0.970 1.000 0.813   
RSMDiff            1.000           0.697 0.757 0.909  0.749 0.967 0.595   
RankSim            1.000           0.869 0.895 0.953  1.000 0.922 0.830   
SVCCA              0.825           0.459 0.597 0.691  0.567 0.973 0.663   
2nd-Cos            1.000           0.949 0.971 0.952  1.000 1.000 0.821   
SoftCorr           0.830           0.683 0.789 0.706  0.867 1.000 0.824 

In [9]:
shortcut_test_df = get_agg_pivot_table("shortcut_test", ["cora", "flickr", "ogbn-arxiv"])
styled = pd.io.formats.style.Styler(
    shortcut_test_df,
    precision=2,
)

latex_str = styled.highlight_max(axis=0, props="textbf:--rwrap;").to_latex("graphs_results_shortcut_test.tex", **LATEX_FORMAT_DICT_PGNN)

#### Augmentation Test Results

In [10]:
get_agg_pivot_table("augmentation_test", ["cora", "flickr", "ogbn-arxiv"])

Measure            AUPRC                                                  \
Dataset             Cora             Flickr             OGBN-Arxiv         
Model                GCN  SAGE   GAT    GCN  SAGE   GAT        GCN  SAGE   
Similarity Measure                                                         
AlignCos           0.510 0.635 0.908  0.686 0.698 0.544      0.736 0.427   
CKA                0.697 0.731 0.906  0.572 0.747 0.508      0.997 1.000   
ConcDiff           0.174 0.504 0.623  0.412 0.348 0.534      0.426 0.533   
DistCorr           0.785 0.732 0.891  0.602 0.794 0.609      1.000 1.000   
EOS                0.535 0.618 0.342  0.679 0.529 0.459      0.819 0.488   
GULP               0.448 0.612 0.349  0.223 0.542 0.555      0.557 0.484   
HardCorr           0.512 0.633 0.513  0.712 0.716 0.571      0.466 0.506   
IMD                0.741 0.343 0.199  0.587 0.566 0.280      1.000 1.000   
Jaccard            0.781 0.807 0.953  0.743 0.880 0.580      1.000 0.991   
LinReg             0.943 0.850 0.872  0.302 0.805 0.338      0.724 0.724   
MagDiff            0.204 0.390 0.535  0.588 0.174 0.430      0.771 1.000   
ShapeMet           0.812 0.784 0.835  0.627 0.760 0.540      0.724 0.724   
OrthProc           0.812 0.784 0.835  0.627 0.760 0.540      0.724 0.724   
PWCCA              0.600 0.650 0.421    NaN 0.567 0.511        NaN 0.477   
PermProc           0.701 0.613 0.423  0.649 0.685 0.398      0.881 0.734   
ProcDist           0.764 0.693 0.749  0.696 0.814 0.401      1.000 1.000   
RSA                0.385 0.573 0.784  0.680 0.718 0.586      0.745 0.495   
RSMDiff            0.987 0.795 1.000  0.644 0.933 0.401      1.000 1.000   
RankSim            0.540 0.618 0.947  0.746 0.882 0.553      1.000 1.000   
SVCCA              0.267 0.294 0.339  0.643 0.673 0.495      0.889 0.813   
2nd-Cos            0.734 0.863 0.967  0.777 0.920 0.622      1.000 0.992   
SoftCorr           0.630 0.702 0.496  0.729 0.581 0.546      0.425 0.460   
UnifDiff           0.747 0.534 1.000  0.683 0.239 0.247      0.757 0.526   

Measure                  Conformity Rate                                 \
Dataset                             Cora             Flickr               
Model                GAT             GCN  SAGE   GAT    GCN  SAGE   GAT   
Similarity Measure                                                        
AlignCos           1.000           0.783 0.881 0.978  0.870 0.901 0.808   
CKA                0.987           0.897 0.919 0.965  0.827 0.912 0.736   
ConcDiff           0.508           0.492 0.781 0.867  0.779 0.681 0.742   
DistCorr           0.989           0.929 0.912 0.965  0.838 0.941 0.812   
EOS                1.000           0.857 0.896 0.752  0.875 0.797 0.703   
GULP               1.000           0.790 0.891 0.763  0.523 0.809 0.839   
HardCorr           0.536           0.849 0.895 0.805  0.894 0.916 0.851   
IMD                1.000           0.931 0.641 0.461  0.871 0.863 0.530   
Jaccard            1.000           0.904 0.946 0.984  0.885 0.973 0.841   
LinReg             0.809           0.983 0.963 0.952  0.527 0.933 0.691   
MagDiff            0.693           0.515 0.742 0.827  0.868 0.449 0.659   
ShapeMet           0.744           0.950 0.931 0.952  0.848 0.937 0.768   
OrthProc           0.744           0.950 0.931 0.952  0.848 0.937 0.768   
PWCCA              0.820           0.859 0.888 0.827    NaN 0.831 0.751   
PermProc           0.936           0.780 0.887 0.514  0.872 0.901 0.616   
ProcDist           1.000           0.885 0.905 0.897  0.887 0.949 0.626   
RSA                0.674           0.681 0.844 0.938  0.857 0.920 0.841   
RSMDiff            1.000           0.997 0.929 1.000  0.873 0.969 0.620   
RankSim            1.000           0.722 0.880 0.983  0.889 0.961 0.835   
SVCCA              0.820           0.543 0.551 0.579  0.838 0.864 0.684   
2nd-Cos            1.000           0.875 0.972 0.989  0.908 0.981 0.836   
SoftCorr           0.448           0.886 0.923 0.796  0.872 0.852 0.837 

In [11]:
augment_test_df = get_agg_pivot_table("augmentation_test", ["cora", "flickr", "ogbn-arxiv"])
styled = pd.io.formats.style.Styler(
    augment_test_df,
    precision=2,
)

latex_str = styled.highlight_max(axis=0, props="textbf:--rwrap;").to_latex("graphs_results_augmentation_test.tex", **LATEX_FORMAT_DICT)

#### Layer Monotonicity Test Results

In [12]:
layer_test_df = get_agg_pivot_table("layer_test", ["cora", "flickr", "ogbn-arxiv"])
styled = pd.io.formats.style.Styler(
    layer_test_df,
    precision=2,
)

latex_str = styled.highlight_max(axis=0, props="textbf:--rwrap;").to_latex("graphs_results_layer_test.tex", **LATEX_FORMAT_DICT_PGNN)

In [13]:
layer_test_df

Measure            Spearman Correlation                                        \
Dataset                            Cora                   Flickr                
Model                               GCN  SAGE   GAT  PGNN    GCN   SAGE   GAT   
Similarity Measure                                                              
AlignCos                          0.980 1.000 1.000 0.313  0.327  0.680 0.040   
CKA                               0.980 1.000 1.000 1.000  0.603  0.890 0.387   
ConcDiff                          0.850 0.250 0.403 0.117  0.383 -0.273 0.243   
DistCorr                          1.000 1.000 1.000 1.000  0.630  0.993 0.467   
EOS                               1.000 1.000 1.000 0.987  1.000  1.000 0.917   
GULP                              1.000 1.000 1.000 0.720  0.540  1.000 0.330   
HardCorr                          0.833 0.983 0.540 0.777  0.627  0.803 0.213   
IMD                               1.000 1.000 1.000 0.887  1.000  0.820 0.970   
Jaccard                           1.000 1.000 1.000 1.000  0.950  0.973 0.953   
LinReg                            1.000 1.000 1.000 1.000  0.450  1.000 0.030   
MagDiff                           0.550 0.490 0.893 0.910  0.583  0.500 0.383   
ShapeMet                          1.000 1.000 1.000 0.910  0.620  0.983 0.453   
OrthProc                          1.000 1.000 1.000 0.910  0.620  0.983 0.453   
PWCCA                             1.000 1.000 1.000 1.000    NaN  1.000 0.373   
PermProc                          0.917 1.000 1.000 1.000  0.603  0.680 0.630   
ProcDist                          0.993 1.000 1.000 0.910  0.660  1.000 0.617   
RSA                               0.837 0.910 1.000 0.607  0.580  0.987 0.437   
RSMDiff                           0.993 1.000 1.000 0.910  0.540  0.653 0.807   
RankSim                           1.000 1.000 1.000 1.000  0.950  0.967 0.987   
SVCCA                             0.410 0.777 0.613 1.000  0.533  0.440 0.467   
2nd-Cos                           1.000 1.000 1.000 0.900  0.810  0.957 0.517   
SoftCorr                          0.957 0.993 0.500 0.753  0.787  0.893 0.113   
UnifDiff                          0.667 0.667 0.590 0.597  0.820  0.890 0.820   

Measure                                    Conformity Rate                    \
Dataset            OGBN-Arxiv                         Cora                     
Model                     GCN   SAGE   GAT             GCN  SAGE   GAT  PGNN   
Similarity Measure                                                             
AlignCos                0.843  0.933 0.837           0.970 1.000 1.000 0.880   
CKA                     0.850  0.457 0.960           0.980 1.000 1.000 1.000   
ConcDiff                0.733  0.390 0.397           0.850 0.520 0.620 0.510   
DistCorr                0.807  0.523 0.933           1.000 1.000 1.000 1.000   
EOS                     1.000  1.000 1.000           1.000 1.000 1.000 0.980   
GULP                    0.733  1.000 1.000           1.000 1.000 1.000 0.850   
HardCorr                0.833  0.620 1.000           0.910 0.990 0.760 0.820   
IMD                     0.550  1.000 1.000           1.000 1.000 1.000 0.950   
Jaccard                 0.973  0.987 1.000           1.000 1.000 1.000 1.000   
LinReg                  0.993  1.000 1.000           1.000 1.000 1.000 1.000   
MagDiff                 0.060  0.290 0.633           0.630 0.720 0.920 0.970   
ShapeMet                0.967  0.993 0.993           1.000 1.000 1.000 0.970   
OrthProc                0.967  0.993 0.993           1.000 1.000 1.000 0.970   
PWCCA                   1.000  1.000 1.000           1.000 1.000 1.000 0.980   
PermProc                0.243  0.883 1.000           0.910 1.000 1.000 1.000   
ProcDist                0.833  0.853 1.000           0.990 1.000 1.000 0.970   
RSA                     0.523  0.973 0.940           0.900 0.970 1.000 0.740   
RSMDiff                 0.850  0.933 1.000           0.990 1.000 1.000 0.970   
RankSim                 0.993  0.987 1.000          

### Results of Output Correlations

In [14]:
def get_output_correlation_table(datasets, corr_func="spearmanr", acc_test = False):
    
    full_csvs = os.listdir(EXPERIMENT_RESULTS_PATH)

    dfs = []
    for dataset in datasets:
        for f in full_csvs:
            if "output" in f and dataset in f:
                df = pd.read_csv(os.path.join(EXPERIMENT_RESULTS_PATH, f))
                df = df.loc[df.loc[:,"quality_measure"] == corr_func]
                data = df.loc[:, ["similarity_measure", "functional_similarity_measure", "corr", "architecture"]]
                data = data.rename(COLUMN_NAME_DICT, axis="columns")
                data["Dataset"] = dataset
                dfs.append(data.iloc[:])

    df_cc = pd.concat(dfs, axis=0)
    df_res = df_cc.pivot_table(index="Similarity Measure", columns=["Measure", "Dataset", "Model"], values="corr", aggfunc="mean")
    df_res = df_res.rename(MEASURE_NAME_DICT, axis="index")
    df_res = df_res.rename(PIVOT_COL_DICT, axis="columns")
    if acc_test: 
        return df_res.loc[:,"AbsoluteAccDiff"]
        
    return df_res.loc[:,["Disagreement", "JSD"]]

In [15]:
output_corr_df = get_output_correlation_table(["cora", "flickr", "ogbn-arxiv"])
styled = pd.io.formats.style.Styler(
    output_corr_df,
    precision=2,
)
styled.highlight_max(axis=0, props="textbf:--rwrap;").to_latex("graphs_results_output_correlation_test.tex", **LATEX_FORMAT_DICT_PGNN)
output_corr_df

Measure            Disagreement                                            \
Dataset                    Cora                      Flickr                 
Model                       GAT    GCN   SAGE   PGNN    GAT    GCN   SAGE   
Similarity Measure                                                          
AlignCos                 -0.051 -0.084  0.271  0.206 -0.083  0.147  0.374   
CKA                       0.445  0.363  0.001  0.137  0.061 -0.211  0.534   
ConcDiff                  0.030 -0.038 -0.097 -0.066  0.029 -0.214 -0.043   
DistCorr                  0.456  0.529 -0.078  0.284 -0.031  0.172  0.395   
EOS                      -0.088  0.127  0.023  0.294  0.231  0.025  0.334   
GULP                     -0.129 -0.165 -0.090  0.334 -0.006  0.029  0.334   
HardCorr                  0.261  0.454  0.104  0.450 -0.093  0.395  0.458   
IMD                      -0.150  0.180 -0.103 -0.135 -0.023 -0.098  0.330   
Jaccard                   0.120  0.436  0.333  0.296 -0.004  0.037  0.419   
LinReg                   -0.062  0.574  0.090  0.228  0.063  0.119  0.457   
MagDiff                  -0.069 -0.073 -0.253  0.250 -0.198  0.056  0.065   
ShapeMet                  0.053  0.616  0.156  0.312  0.034  0.193  0.573   
OrthProc                  0.053  0.616  0.156  0.312  0.034  0.192  0.573   
PWCCA                    -0.138  0.081  0.360  0.212  0.159    NaN  0.318   
PermProc                 -0.138  0.235  0.328  0.422 -0.272  0.178 -0.097   
ProcDist                 -0.175  0.450  0.237  0.427 -0.265 -0.134 -0.148   
RSA                      -0.018  0.055  0.328  0.479  0.063  0.348  0.586   
RSMDiff                  -0.150  0.050  0.137  0.345 -0.261 -0.297 -0.003   
RankSim                   0.442  0.343  0.227  0.303  0.047  0.055  0.237   
SVCCA                     0.166 -0.057  0.045  0.004  0.028 -0.269  0.214   
2nd-Cos                   0.299  0.399  0.252  0.153 -0.103  0.301  0.143   
SoftCorr                  0.069  0.485  0.057  0.405  0.017  0.390  0.486   
UnifDiff                 -0.146  0.067  0.161  0.276  0.182 -0.340  0.038   

Measure                                        JSD                       \
Dataset            OGBN-Arxiv                 Cora                        
Model                     GAT    GCN   SAGE    GAT    GCN   SAGE   PGNN   
Similarity Measure                                                        
AlignCos                0.166 -0.098  0.001  0.174  0.017  0.384 -0.143   
CKA                     0.227  0.029 -0.041  0.535  0.435 -0.031 -0.327   
ConcDiff               -0.160 -0.254  0.072  0.014  0.032  0.041  0.205   
DistCorr                0.204  0.110  0.081  0.601  0.597  0.053 -0.362   
EOS                     0.119 -0.255 -0.023 -0.036  0.168  0.224 -0.225   
GULP                    0.099  0.154 -0.037 -0.042 -0.096  0.182  0.100   
HardCorr                0.239  0.020 -0.241  0.516  0.479  0.162 -0.174   
IMD                     0.037 -0.125  0.012 -0.086  0.213 -0.404  0.104   
Jaccard                 0.219  0.142  0.012  0.379  0.441  0.453  0.051   
LinReg                  0.221 -0.099 -0.188  0.190  0.580  0.331 -0.267   
MagDiff                 0.216 -0.252 -0.195 -0.145  0.004  0.063  0.101   
ShapeMet                0.268  0.018 -0.165  0.282  0.695  0.272 -0.251   
OrthProc                0.268  0.018 -0.165  0.282  0.695  0.272 -0.251   
PWCCA                   0.033 -0.243 -0.079  0.015  0.080  0.267 -0.270   
PermProc                0.221  0.250 -0.412  0.133  0.289  0.096  0.006   
ProcDist                0.303  0.257 -0.382  0.046  0.516  0.085  0.012   
RSA                     0.454  0.170  0.155  0.200  0.117  0.133  0.106   
RSMDiff                 0.356 -0.141  0.090 -0.072  0.115 -0.011  0.006   
RankSim                 0.096  0.027  0.274  0.505  0.363  0.561 -0.042   
SVCCA                  -0.146  0.085  0.113  0.261 -0.085  0.018 -0.027   
2nd-Cos                 0.064  0.042  0.265  0.529  0.429  0.536 -0.240   
SoftCorr                0.353  0.034 -0.260  

In [16]:
acc_corr_df = get_output_correlation_table(["cora", "flickr", "ogbn-arxiv"], acc_test=True)
styled = pd.io.formats.style.Styler(
    acc_corr_df,
    precision=2,
)
styled.highlight_max(axis=0, props="textbf:--rwrap;").to_latex("graphs_results_accuracy_correlation_test.tex", **LATEX_FORMAT_DICT_PGNN)
acc_corr_df

Dataset              Cora                      Flickr                \
Model                 GAT    GCN   SAGE   PGNN    GAT    GCN   SAGE   
Similarity Measure                                                    
AlignCos           -0.289 -0.332  0.132  0.050 -0.067  0.354  0.237   
CKA                 0.023  0.163 -0.172 -0.009 -0.160  0.025  0.267   
ConcDiff           -0.197  0.154 -0.252  0.354 -0.071 -0.079 -0.287   
DistCorr            0.026  0.008 -0.175  0.153 -0.193  0.413  0.416   
EOS                -0.049 -0.236  0.080  0.272  0.290  0.149 -0.265   
GULP               -0.121 -0.435  0.081  0.276 -0.273  0.036 -0.271   
HardCorr           -0.143  0.106 -0.109  0.322  0.059  0.306  0.352   
IMD                -0.023 -0.114 -0.005 -0.157 -0.089 -0.095  0.366   
Jaccard            -0.160  0.023 -0.111  0.079 -0.180  0.317  0.281   
LinReg             -0.109  0.058 -0.207  0.061 -0.180 -0.036  0.174   
MagDiff            -0.192  0.077 -0.133  0.357  0.145  0.023 -0.173   
ShapeMet           -0.118  0.147 -0.292  0.202 -0.147  0.395  0.280   
OrthProc           -0.118  0.147 -0.292  0.202 -0.147  0.394  0.280   
PWCCA              -0.261 -0.050  0.068  0.085 -0.163    NaN -0.051   
PermProc           -0.261 -0.118  0.181  0.475  0.150  0.205 -0.195   
ProcDist           -0.192  0.083  0.006  0.475  0.113  0.016 -0.065   
RSA                -0.279 -0.271  0.043  0.375 -0.082  0.531  0.316   
RSMDiff            -0.141 -0.189  0.074  0.448  0.126 -0.184 -0.161   
RankSim             0.321 -0.004 -0.103  0.068 -0.198  0.346  0.312   
SVCCA              -0.186 -0.015 -0.126 -0.015 -0.176  0.007  0.015   
2nd-Cos            -0.109 -0.100  0.043  0.150  0.009  0.544 -0.187   
SoftCorr            0.026  0.183 -0.053  0.346 -0.075  0.301  0.331   
UnifDiff           -0.088 -0.103 -0.037 -0.012 -0.183 -0.175  0.034   

Dataset            OGBN-Arxiv                
Model                     GAT    GCN   SAGE  
Similarity Measure                           
AlignCos               -0.172 -0.081  0.168  
CKA                    -0.052 -0.167  0.106  
ConcDiff               -0.117 -0.073 -0.133  
DistCorr               -0.060 -0.098  0.146  
EOS                    -0.324 -0.213  0.054  
GULP                   -0.345 -0.084  0.061  
HardCorr                0.040  0.366  0.015  
IMD                    -0.148 -0.207 -0.020  
Jaccard                -0.145 -0.321 -0.130  
LinReg                 -0.188  0.070 -0.011  
MagDiff                 0.109 -0.180 -0.198  
ShapeMet               -0.086 -0.035  0.090  
OrthProc               -0.086 -0.035  0.090  
PWCCA                  -0.296 -0.091  0.059  
PermProc                0.426 -0.094  0.032  
ProcDist                0.430 -0.174  0.069  
RSA                     0.317 -0.071  0.248  
RSMDiff                 0.019 -0.046 -0.194  
RankSim                -0.090 -0.277  0.049  
SVCCA                  -0.100 -0.268  0.095  
2nd-Cos                -0.189 -0.471  0.224  
SoftCorr                0.122  0.354  0.121  
UnifDiff               -0.250 -0.187 -0.203